## Lab 2 Termodinámica y Teoría Cinética 🧑‍🔬 - Conductividad Térmica

- Camila Ferrus
- Marcelo Ojeda

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import Symbol

In [2]:
err_pesa_g = 0.005
err_piedemetro_mm = 0.5
err_cronometro_s = 0.005
err_termometro_celsius = 0.5

espesor_termopanel_mm = 16
espesor_vidrio_mm = 5

### Manejo de errores

Podríamos ignorar el error instrumental de la pesa y el cronómetro, ya que son muy pequeños en comparación a los errores aleatorios.

Los errores se combinan de acuerdo a las fórmulas de la guía de análisis de errores de la página de los laboratorios.

### Calcular el cambio de calor con respecto al tiempo

Utilizaremos la siguiente fórmula:

$$ \dot Q =\frac{\Delta Q}{\Delta t} = \frac{q_s \cdot m}{\Delta t} $$

Donde $ q_s = 334 \frac{kJ}{kg}$ es el calor latente de fusión, $ m $ la masa y $ \Delta t $ un intervalo de tiempo.

### Calcular el coeficiente de conductividad térmica

Utilizaremos la siguiente fórmula:

\begin{align}
\dot Q &= \frac{\Delta Q}{\Delta t} = kA\frac{\Delta T}{h} \\
\Longrightarrow k &= \dot Q \frac{h}{A \Delta T} \\
\end{align}


Donde,

- $\Delta Q$: es el calor intercambiado en el tiempo, entre el foco frío y el caliente.
- $\Delta T$: diferencia de temperatura entre los focos frío y caliente

- $k$: conductividad térmica (es el parámetro que buscamos)

- $A$: área de la sección transversal del bloque de hielo

- $h$: espesor de la muestra.



In [3]:
def area_transversal_bloque(diametro_1, diametro_2):
    area_1 = round(2 * np.pi * (diametro_1 / 2), 1)
    area_2 = round(2 * np.pi * (diametro_2 / 2), 1)
    
    area = round(np.sqrt(((area_1 + area_2) / 2) ** 2), 2)
    area_err = round(np.sqrt(((area_1 - area_2) / 2) ** 2), 2)
    
    return 'Área de la sección transversal del bloque: {} ± {} mm2'.format(area,
                                                                          area_err)

def encontrar_cambio_calor(masa, material: str):  # la masa se entrega en gramos, la función hace la conversión a kilos 
    
    masa_kg = masa / 1000
    calor = round(334 * masa_kg, 2)
    
    dq_dm = 334
    calor_err = round(np.sqrt((dq_dm * (0.005 / 1000)) ** 2), 3)
    
    return 'Cambio de calor para {}: {} ± {} kJ/°C'.format(material, calor, calor_err)

def encontrar_coeficiente(cambio_calor_kj, cambio_calor_err_kj, 
                          espesor, espesor_err,
                          area, area_err,
                          cambio_temperatura, cambio_temperatura_err,
                          material: str):
    
    area_cm2 = area / 100
    espesor_cm = espesor / 10
    cambio_calor = cambio_calor_kj * 1000
    cambio_calor_err = cambio_calor_err_kj * 1000
    
    coef = round(((cambio_calor * (espesor_cm / (area_cm2 * cambio_temperatura))) / 600), 2)   # en un intervalo de 10 min
    
    dk_dQ = (espesor_cm / (area_cm2 * cambio_temperatura))
    err_calor = round(np.sqrt((dk_dQ * cambio_calor_err) ** 2), 3)
    
    dk_dh = (cambio_calor / (area_cm2 * cambio_temperatura))
    err_espesor = round(np.sqrt((dk_dh * (espesor_err / 10)) ** 2), 3)
    
    dk_dA = - cambio_calor * (espesor_cm / ((area_cm2 ** 2) * cambio_temperatura))
    err_area = round(np.sqrt((dk_dA * (area_err / 100)) ** 2), 3)
    
    dk_dT = - cambio_calor * (espesor_cm / (area_cm2 * (cambio_temperatura ** 2)))
    err_temp = round(np.sqrt((dk_dT * cambio_temperatura_err) ** 2), 3)
    
    err_total = round(((err_calor + err_espesor + err_area + err_temp) / 600), 3)
    
    return 'Coeficiente de conductividad térmica para {}: {} ± {} W/cm°C'.format(material, coef, err_total)
    
    
    
    

## Termopanel

In [4]:
d1_mm_tp = 76 # diámetro del hielo al principio
d2_mm_tp = 71 # diámetro del hielo después de ponerle el vapor

t_amb_celsius_tp = [23,23,23,23,23,23,23,23,23,22,22,22,22,22,22,22,22,21,21,21,21,21,21,20,20,21,20,20,20,20,20,20,19,19,19,19] # temperaturas ambiente
t_vapor_celsius_tp = [62,62,61,62,62,62,62,62,62,61,61,61,65,65,67,67,67,67,67,68,67,65,67,67,65,65,65,67,67,67,67,67,67,67,67,
                   67,67,68,68,68,69,69,69,69,70,70,70,70,70,70,71,70,71,71,71,72,72,73,72,72,72,72,73,73,74,74,74,74,73,74,
                   73,73,73,74,74,74,74,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,
                   75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,  # temperaturas con vapor
                   75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,
                   75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,
                   75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75]

t_amb_median_tp = round(np.median(t_amb_celsius_tp), 1)                             # mediana de las temperaturas ambiente
t_amb_std_tp = round(np.std(t_amb_celsius_tp, ddof=1), 1) + err_termometro_celsius  # desviación estándar de las temperaturas ambiente

t_vapor_median_tp = round(np.median(t_vapor_celsius_tp), 1)                              # mediana de las temperaturas con vapor
t_vapor_std_tp = round(np.std(t_vapor_celsius_tp, ddof=1), 1) + err_termometro_celsius   # desviación estándar de las temperaturas con vapor


masa_vaso_g_tp = 102.37  # masa del vaso vacío

m1_tp = round(117.39 - masa_vaso_g_tp, 2)   # agua obtenida a temperatura ambiente
m2_tp = round(122.60 - masa_vaso_g_tp, 2)   # agua obtenida a temperatura con vapor

print(' ' *7, 'Termopanel\n', '-' *28)
print(f'Agua obtenida a temperatura ambiente: {m1_tp} g')
print(f'Agua obtenida a temperatura con vapor: {m2_tp} g')
print('Diferencia de temperatura: {} ± {} °C'.format(t_vapor_median_tp - t_amb_median_tp,
                                                     round(t_vapor_std_tp + t_amb_std_tp), 1))

        Termopanel
 ----------------------------
Agua obtenida a temperatura ambiente: 15.02 g
Agua obtenida a temperatura con vapor: 20.23 g
Diferencia de temperatura: 54.0 ± 6 °C


In [5]:
area_bloque_tp = area_transversal_bloque(d1_mm_tp, d2_mm_tp)
print(area_bloque_tp)

Área de la sección transversal del bloque: 230.95 ± 7.85 mm2


In [6]:
cambio_calor_tp = encontrar_cambio_calor(m2_tp - m1_tp, 'termopanel')
print(cambio_calor_tp)

Cambio de calor para termopanel: 1.74 ± 0.002 kJ/°C


In [8]:
coef_tp = encontrar_coeficiente(cambio_calor_kj= 1.74, cambio_calor_err_kj=0.002,
                                espesor= espesor_termopanel_mm, espesor_err= err_piedemetro_mm,
                                area= 230.95, area_err=7.85,
                                cambio_temperatura= 54.0, cambio_temperatura_err=6,
                                material='termopanel')
print(coef_tp)

Coeficiente de conductividad térmica para termopanel: 0.04 ± 0.007 W/cm°C


## Vidrio

In [9]:
d1_mm_vd = 68  # diámetro del hielo al principio
d2_mm_vd = 63  # diámetro del hielo después de ponerle el vapor

t_amb_celsius_vd = [23,23,23,23,23,23,23,23,23,22,22,22,22,22,22,22,22,21,21,21,21,21,21,20,20,21,20,20,20,20,20,20,19,19,19,19]  # temperaturas ambiente
t_vapor_celsius_vd = [48,48,49,48,48,48,48,48,48,49,49,49,50,50,51,51,51,51,51,52,51,50,51,51,50,50,50,51,51,51,51,51,51,51,51,
                   51,51,52,52,52,53,53,53,53,54,54,54,54,54,54,55,54,55,55,55,56,56,57,56,56,56,56,57,57,58,58,58,58,57,58,
                   57,57,57,58,58,58,58,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,   # temperaturas con vapor
                   59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,
                   59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,
                   59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,
                   59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59]

t_amb_median_vd = round(np.median(t_amb_celsius_vd), 1)                             # mediana de las temperaturas ambiente
t_amb_std_vd = round(np.std(t_amb_celsius_vd, ddof=1), 1) + err_termometro_celsius  # desviación estándar de las temperaturas ambiente

t_vapor_median_vd = round(np.median(t_vapor_celsius_vd), 1)                              # mediana de las temperaturas con vapor
t_vapor_std_vd = round(np.std(t_vapor_celsius_vd, ddof=1), 1) + err_termometro_celsius   # desviación estándar de las temperaturas con vapor

masa_vaso_vd = 102.24  # masa del vaso vacío

m1_vd = round(118.00 - masa_vaso_vd, 2)  # agua obtenida a temperatura ambiente
m2_vd = round(159.88 - masa_vaso_vd, 2)  # agua obtenida a temperatura con vapor

In [10]:
print(' ' *7, 'Vidrio\n', '-' *28)
print(f'Agua obtenida a temperatura ambiente: {m1_vd} g')
print(f'Agua obtenida a temperatura con vapor: {m2_vd} g')
print('Diferencia de temperatura: {} ± {} °C'.format(t_vapor_median_vd - t_amb_median_vd,
                                                     round(t_vapor_std_vd + t_amb_std_vd), 1))

        Vidrio
 ----------------------------
Agua obtenida a temperatura ambiente: 15.76 g
Agua obtenida a temperatura con vapor: 57.64 g
Diferencia de temperatura: 38.0 ± 6 °C


In [11]:
area_bloque_tp = area_transversal_bloque(d1_mm_vd, d2_mm_vd)
print(area_bloque_tp)

Área de la sección transversal del bloque: 205.75 ± 7.85 mm2


In [12]:
cambio_calor_tp = encontrar_cambio_calor(m2_vd - m1_vd, 'termopanel')
print(cambio_calor_tp)

Cambio de calor para termopanel: 13.99 ± 0.002 kJ/°C


In [13]:
coef_tp = encontrar_coeficiente(cambio_calor_kj= 13.99, cambio_calor_err_kj=0.002,
                                espesor= espesor_vidrio_mm, espesor_err= err_piedemetro_mm,
                                area= 205.75, area_err=7.85,
                                cambio_temperatura= 38.0, cambio_temperatura_err=6,
                                material='vidrio')
print(coef_tp)

Coeficiente de conductividad térmica para vidrio: 0.15 ± 0.044 W/cm°C
